# Visualizations for IPL data 2008 - 2019

In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import data

In [ ]:
# importing the crawled data which are in csv files
data_folder = 'data/'
matches = pd.read_csv(data_folder + 'match_results.csv')

In [ ]:
matches.head()

# Pre-processing data

In [ ]:
# Replacing team names with their latest ones
# as some teams changed their name in between
matches.replace('Rising Pune Supergiants', 'Rising Pune Supergiant', inplace=True)
matches.replace('Delhi Daredevils', 'Delhi Capitals', inplace=True)
print('All teams: ', matches['team1'].unique())
print('Total number of teams(2008-2019): ', len(matches['team1'].unique()))

# Toss analysis

In [ ]:
team = 'Sunrisers Hyderabad'
# related matches
rltd_mtchs = matches[(matches['team1']==team) | (matches['team2']==team)]
# rltd_mtchs

In [ ]:
# toss_decisions = matches[['toss_winner', 'toss_decision', 'id']].groupby(['toss_winner', 'toss_decision']).count()
toss_decisions = matches[['toss_winner', 'toss_decision', 'id']].\
                        groupby(['toss_winner', 'toss_decision']).agg({'id': 'count'})

toss_decisions.head(3)
toss_decisions = toss_decisions.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
toss_decisions

In [ ]:
# toss_decisions.plot.bar()
toss_decisions.unstack().plot(kind='bar', rot=45)

In [ ]:
toss_decisions.unstack()